In [1]:
import os
import numpy as np
import trimesh
import pyrender
import matplotlib.pyplot as plt

# Environment setup for offscreen rendering
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["LIBGL_ALWAYS_SOFTWARE"] = "1"

# Configuration
INPUT_ROOT = "/home/jeans/win/aaaJAIST/resources/LOD_data_50"
OUTPUT_ROOT = "/home/jeans/progressive_img2sketch/resources/LOD_images"
LODS = [1, 2, 3]
SCENES = range(1, 51)  # scene_number from 0 to 50
AZIMUTH_STEP = 30      # degrees between each camera azimuth
ELEVATIONS = [0] #[0, 15, 30]  # camera elevations in degrees
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080


def center_scene_by_bbox(scene: trimesh.Scene) -> trimesh.Scene:
    """
    Center the scene at the origin based on its bounding-box center.
    """
    min_corner, max_corner = scene.bounds
    center = (min_corner + max_corner) / 2.0
    scene.apply_translation(-center)
    return scene


def get_registration_matrix(
    source_mesh: trimesh.Trimesh,
    target_mesh: trimesh.Trimesh,
    samples: int = 3000,
    icp_first: int = 1,
    icp_final: int = 30
) -> np.ndarray:
    """
    Compute the ICP transformation matrix that aligns source_mesh to target_mesh.
    """
    matrix, _ = trimesh.registration.mesh_other(
        source_mesh,
        target_mesh,
        samples=samples,
        scale=False,
        icp_first=icp_first,
        icp_final=icp_final
    )
    return matrix


def align_lods(scenes: dict[int, trimesh.Scene], center_before: bool = False):
    # — step 1: (optional) rough centering to help ICP converge —
    if center_before:
        for lod in scenes:
            scenes[lod] = center_scene_by_bbox(scenes[lod])

    # — step 2: extract single meshes for ICP —
    meshes = {
        lod: trimesh.util.concatenate(list(scenes[lod].geometry.values()))
        for lod in scenes
    }

    #show original bbox centers
    for lod, mesh in meshes.items():
        min_corner, max_corner = mesh.bounds
        center = (min_corner + max_corner) / 2.0
        print(f"LOD {lod} original center: {center}")
        
    # ICP: 2→1 then 3→2
    t2_1 = get_registration_matrix(meshes[2], meshes[1])
    t3_2 = get_registration_matrix(meshes[3], meshes[2])

    # apply those transforms
    scenes[2].apply_transform(t2_1)
    scenes[3].apply_transform(t2_1 @ t3_2)

    # show aligned bbox centers
    for lod, scene in scenes.items():
        min_corner, max_corner = scene.bounds
        center = (min_corner + max_corner) / 2.0
        print(f"LOD {lod} aligned center: {center}")
    # — step 3: **final centering** based on aligned LOD1 bbox —
    min1, max1 = scenes[1].bounds
    center1 = (min1 + max1) * 0.5
    for lod in scenes:
        scenes[lod].apply_translation(-center1)

    return scenes



def look_at_matrix(eye: np.ndarray, target: np.ndarray, up: np.ndarray) -> np.ndarray:
    """
    Create a camera-to-world pose matrix for pyrender given eye, target, up vectors.
    """
    f = (target - eye)
    f /= np.linalg.norm(f)
    # avoid parallel up/f
    if np.isclose(np.linalg.norm(np.cross(f, up)), 0):
        up = np.array([0, 0, 1]) if np.isclose(abs(f.dot([0, 1, 0])), 1) else np.array([0, 1, 0])
    s = np.cross(f, up); s /= np.linalg.norm(s)
    u = np.cross(s, f); u /= np.linalg.norm(u)

    # view matrix (world→camera)
    view = np.array([
        [ s[0],  s[1],  s[2], -s.dot(eye)],
        [ u[0],  u[1],  u[2], -u.dot(eye)],
        [-f[0], -f[1], -f[2],  f.dot(eye)],
        [    0,     0,     0,           1]
    ])
    # invert → camera pose (camera→world)
    return np.linalg.inv(view)

SHOW_Y_AXIS = True  # Set to True to visualize the Y-axis

def render_orbit(scenes: dict[int, trimesh.Scene], scene_number: int):
    """
    For each LOD scene, render images at multiple azimuths and elevations and save.
    """
    # Use LOD3 to determine orbit radius
    bounds = scenes[3].extents
    radius = np.max(bounds) / 2.0
    target = np.array([0.0, 0.0, 0.0])

    renderer = pyrender.OffscreenRenderer(IMAGE_WIDTH, IMAGE_HEIGHT)
    for lod, scene in scenes.items():
        pyrender_scene = pyrender.Scene.from_trimesh_scene(scene)
        # add a uniform ambient term (rgb)
        # pyrender_scene.ambient_light = np.array([0.05, 0.05, 0.05])
            # ←── insert SHOW_Y_AXIS here ──→
        if SHOW_Y_AXIS:
            # a very tall, thin box along Y to visualize the Y-axis
            bbox = trimesh.primitives.Box(extents=[1, max(scenes[3].extents)*2, 1])
            mat  = pyrender.Material(wireframe=True)
            mesh = pyrender.Mesh.from_trimesh(bbox, material=mat)
            pyrender_scene.add(mesh)
        
        # ← insert the wireframe overlay for LOD1 here
        # if lod == 1:
        #     combined = trimesh.util.concatenate(list(scene.geometry.values()))
        #     wire_mat = pyrender.Material( wireframe=True)
        #     wireframe = pyrender.Mesh.from_trimesh(combined, material=wire_mat, smooth=False)
        #     pyrender_scene.add(wireframe)
        
        # —– add Raymond lighting —–
        intensity = 3.0

        key = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity)
        key_pose = np.array([
            [ 0,  0,  1,  2],
            [ 0,  1,  0,  2],
            [ 1,  0,  0,  2],
            [ 0,  0,  0,  1],
        ])
        pyrender_scene.add(key, pose=key_pose)

        fill = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity * 0.5)
        fill_pose = np.array([
            [ 0,  0, -1, -2],
            [ 0,  1,  0,  1],
            [-1,  0,  0, -2],
            [ 0,  0,  0,  1],
        ])
        pyrender_scene.add(fill, pose=fill_pose)

        back = pyrender.DirectionalLight(color=np.ones(3), intensity=intensity * 0.3)
        back_pose = np.array([
            [ 1,  0,  0, -2],
            [ 0,  0,  1, -2],
            [ 0,  1,  0,  2],
            [ 0,  0,  0,  1],
        ])
        pyrender_scene.add(back, pose=back_pose)
        # —– end lights —–


        for az in range(0, 360, AZIMUTH_STEP):
            for el in ELEVATIONS:
                # spherical → cartesian
                rad_az = np.deg2rad(az)
                rad_el = np.deg2rad(el)
                x = radius * 2 * np.cos(rad_el) * np.sin(rad_az)
                y = radius * 2 * np.sin(rad_el)
                z = radius * 2 * np.cos(rad_el) * np.cos(rad_az)
                eye = np.array([x, y, z])

                # setup camera
                cam_pose = look_at_matrix(eye, target, np.array([0, 1, 0]))
                camera = pyrender.PerspectiveCamera(yfov=np.pi/3.0, aspectRatio=IMAGE_WIDTH/IMAGE_HEIGHT)
                cam_node = pyrender_scene.add(camera, pose=cam_pose)

                # render
                color, _ = renderer.render(pyrender_scene)
                pyrender_scene.remove_node(cam_node)

                # save
                save_dir = os.path.join(OUTPUT_ROOT, str(scene_number), str(lod), str(az), str(el))
                os.makedirs(save_dir, exist_ok=True)
                file_name = f"{scene_number}_{lod}_{az}_{el}.png"
                plt.imsave(os.path.join(save_dir, file_name), color)

    renderer.delete()


def process_dataset():
    """
    Iterate through all scenes and LODs, align them, then render orbits.
    """
    for scene_num in SCENES:
        # load scenes
        lod_scenes = {
            lod: trimesh.load(os.path.join(INPUT_ROOT, str(scene_num), f"lod{lod}.obj"))
            for lod in LODS
        }
        # align LODs
        aligned_scenes = align_lods(lod_scenes)
        
        # return aligned_scenes
        # render and save images
        render_orbit(aligned_scenes, scene_num)


# if __name__ == "__main__":
#     process_dataset()


In [2]:
aligned_scene = process_dataset()

LOD 1 original center: [ 738.909   12.6   -976.684]
LOD 2 original center: [ 738.794     14.35638 -678.243  ]
LOD 3 original center: [ 738.969      14.630361 -377.992   ]
LOD 1 aligned center: [ 738.909   12.6   -976.684]
LOD 2 aligned center: [ 738.44289607   14.00997509 -979.2272134 ]
LOD 3 aligned center: [ 738.54757945   13.93744896 -979.23634617]
LOD 1 original center: [1615560.            6543.54999999 -697738.5       ]
LOD 2 original center: [1318715.            6899.99999936 -654347.        ]
LOD 3 original center: [1021880.5      6502.022 -610955.   ]
LOD 1 aligned center: [1615560.            6543.54999999 -697738.5       ]
LOD 2 aligned center: [1637301.66601181    7252.70834019 -687296.77806878]
LOD 3 aligned center: [1637295.58255918    6849.91045018 -687295.82715246]
LOD 1 original center: [1068537.5     10927.504 -192875.15 ]
LOD 2 original center: [ 768535.      12727.504 -192875.15 ]
LOD 3 original center: [ 468535.      12727.504 -192875.15 ]


/home/jeans/miniconda3/envs/3dline/lib/python3.10/site-packages/trimesh/triangles.py:656: RuntimeWarning: invalid value encountered in divide
  v = (d1[is_ab] / (d1[is_ab] - d3[is_ab])).reshape((-1, 1))


LOD 1 aligned center: [1068537.5     10927.504 -192875.15 ]
LOD 2 aligned center: [1068774.71035013    9422.49239907 -192197.31959204]
LOD 3 aligned center: [1068797.32074719   10011.38420833 -192219.69573785]
LOD 1 original center: [2015.84        12.6497506  664.927    ]
LOD 2 original center: [1619.135       13.3997506  664.927    ]
LOD 3 original center: [1219.135       13.3997506  664.759    ]
LOD 1 aligned center: [2015.84        12.6497506  664.927    ]
LOD 2 aligned center: [2018.04263112   13.1101547   665.06234642]
LOD 3 aligned center: [2017.99418873   12.68206495  664.88129938]
LOD 1 original center: [ 963803.     7900.  -424571.5]
LOD 2 original center: [ 763805.       7698.272 -424571.5  ]
LOD 3 original center: [ 563805.       8498.272 -424571.5  ]
LOD 1 aligned center: [ 963803.     7900.  -424571.5]
LOD 2 aligned center: [ 963823.46829345    5811.43052145 -423848.74552343]
LOD 3 aligned center: [ 963809.88897222    6536.3238097  -423854.03182245]
LOD 1 original center:

KeyboardInterrupt: 

In [ ]:
# aligned_scene[1].show()

# test_scene = trimesh.Scene()
# # turn all 3 scenes to trimesh and show it in trimesh show
# test_scene.add_geometry(aligned_scene[1])
# test_scene.add_geometry(aligned_scene[2])
# test_scene.add_geometry(aligned_scene[3])
# test_scene.show()
